In [1]:
from google.colab import drive 
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [62]:
!cp -av 'drive/MyDrive/CV/final_project/data_pure.zip' './' 
!unzip -u -q "data_pure.zip"

'drive/MyDrive/CV/final_project/data_pure.zip' -> './data_pure.zip'


In [65]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

seed = 1
base_dir = './'
df = pd.read_csv(base_dir+"data/train.csv")
df['filename'] = df.apply(lambda row: row['id']+'.jpg',axis=1)

data_gen_args = dict(rescale=1./255,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0,
                     horizontal_flip=False,
                     vertical_flip=False,
                     validation_split=0.1)

train_datagen = ImageDataGenerator(**data_gen_args)
val_datagen = ImageDataGenerator(**data_gen_args)

train_generator = train_datagen.flow_from_dataframe(dataframe=df,
                                                    directory=base_dir+'data/train/',
                                                    x_col='filename',
                                                    y_col=['x','y'],
                                                    subset="training",
                                                    batch_size = 32,
                                                    seed=seed,
                                                    shuffle=True,
                                                    class_mode='raw',
                                                    target_size=(224,224))
val_generator = val_datagen.flow_from_dataframe(dataframe=df,
                                                directory=base_dir+'data/train/',
                                                x_col='filename',
                                                y_col=['x','y'],
                                                subset="validation",
                                                batch_size = 32,
                                                seed=seed,
                                                shuffle=True,
                                                class_mode='raw',
                                                target_size=(224,224))

Found 6750 validated image filenames.
Found 750 validated image filenames.


In [27]:
df_test = pd.read_csv("data/imagenames.csv")
df_test['filename'] = df_test.apply(lambda row: row['id']+'.jpg',axis=1)
df_test['x']=0
df_test['y']=0
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(dataframe=df_test,
                                                directory=base_dir+'data/test/',
                                                x_col='filename',
                                                y_col=['x','y'],
                                                batch_size = 32,
                                                shuffle=False,
                                                class_mode='raw',
                                                target_size=(224,224))

Found 1200 validated image filenames.


In [38]:
from tensorflow.keras.applications import VGG16,Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten, Dense, Dropout
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.regularizers import l2

In [40]:
def location_regressor():
    Xception_model = Xception(input_shape=(224,224,3),include_top=False,weights='imagenet')
    for layer in Xception_model.layers:
        layer.trainable = False
    x = Flatten()(Xception_model.output)
    x=Dense(1048,activation='relu')(x)
    x=Dense(128,activation='relu')(x)
    x=Dense(2)(x)
    model = Model(inputs=Xception_model.input, outputs=x)
    model.compile(optimizer='Adam',loss=MeanAbsoluteError(),metrics=['mean_absolute_error'])
    return model

In [58]:
with tf.device('/device:GPU:0'):
    model = location_regressor()
    #model.load_weights('drive/MyDrive/CV/Model/regressor.h5')
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
    r=model.fit(train_generator, validation_data=val_generator,
                validation_steps=20,steps_per_epoch=211,epochs=100,callbacks=[callback])

Epoch 1/100
235/235 [==============================] - 121s 505ms/step - loss: 5.9204 - mean_absolute_error: 5.9204 - val_loss: 7.7783 - val_mean_absolute_error: 7.7783
Epoch 2/100
235/235 [==============================] - 118s 502ms/step - loss: 5.8272 - mean_absolute_error: 5.8272 - val_loss: 7.0879 - val_mean_absolute_error: 7.0879
Epoch 3/100
235/235 [==============================] - 117s 496ms/step - loss: 5.8523 - mean_absolute_error: 5.8523 - val_loss: 6.9393 - val_mean_absolute_error: 6.9393
Epoch 4/100
235/235 [==============================] - 114s 487ms/step - loss: 5.6054 - mean_absolute_error: 5.6054 - val_loss: 7.2219 - val_mean_absolute_error: 7.2219
Epoch 5/100
235/235 [==============================] - 116s 496ms/step - loss: 5.5295 - mean_absolute_error: 5.5295 - val_loss: 7.0352 - val_mean_absolute_error: 7.0352


In [59]:
model.save('drive/MyDrive/CV/Model/regressor.h5')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [60]:
with tf.device('/device:GPU:0'):
    ypred = model.predict(test_generator)
ypred.shape

(1200, 2)

In [61]:
df_test['x'] = ypred[:,0]
df_test['y'] = ypred[:,1]
if 'filename' in df_test.keys():
    df_test=df_test.drop(columns=['filename'])
df_test.head()
df_test.to_csv('pred_regression.csv',index=False)